# Import Package

In [1]:
import time
import numpy as np 
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

# (1.) Selenium Path and Web

In [2]:
path   = r"/Users/chen-lichiang/Desktop/chromedriver" # your selenium chrome driver path
driver = webdriver.Chrome(path)

In [3]:
url = r'https://www.sitca.org.tw/ROC/Industry/IN4001.aspx?PGMID=IN0401'
driver.get(url)

#  (2.) Select 月份

In [4]:
from selenium.webdriver.support.ui import Select

select = Select(driver.find_element_by_xpath("//select[@name ='ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolder1$ddlYYYYMM']"))
select.select_by_value("202109")

# (3.) Beautifulsoup

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>SITCA</title>
<link charset="utf-8" href="image/ico_sitca.ico" rel="shortcut icon" type="image/vsd.microsoft.ico"/>
<link href="../../css/table1.css" rel="stylesheet" type="text/css"/>
<link href="../../css/MeunTab.css" rel="stylesheet" type="text/css"/>
<link href="../../css/mod_settings.css" rel="stylesheet" type="text/css"/>
<!-- include jQuery library -->
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.4.1/jquery.min.js" type="text/javascript"></script>
<script src="../../lib/jquerycssmenu.js" type="text/javascript"></script>
<script src="../../lib/fade_pic_rotator.js" type="text/javascript"></script>
<script src="../../lib/tabmenu.js" type="text/javascript"></script>
<!--[if lte IE 8]>
        <style type="text/css">
            html .jquerycssmenu{height: 1%

In [6]:
type(soup)

bs4.BeautifulSoup

# (4.) Beautiful Soup Locate desire table

In [7]:
soup.find('table',{ 'id' : 'ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_TableCarte'})

<table border="0" border-collapse="collapse" border-color="#CCCCCC" id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_TableCarte" width="100%">
<tbody><tr>
<td class="MemDataBlueHeader" style="text-align:Center;">委任人類別</td><td class="MemDataBlueHeader" style="text-align:Center;">契約數量</td><td class="MemDataBlueHeader" style="text-align:Center;">全體有效契約金額</td>
</tr><tr>
<td class="r_blue" style="text-align:Left;">本國自然人</td><td class="r_blue" style="text-align:Right;">247</td><td class="r_blue" style="text-align:Right;">16,279,451,706</td>
</tr><tr>
<td class="r_blue" style="text-align:Left;">外國自然人</td><td class="r_blue" style="text-align:Right;">0</td><td class="r_blue" style="text-align:Right;">0</td>
</tr><tr>
<td class="r_blue" style="text-align:Left;">本國法人</td><td class="r_blue" style="text-align:Right;">168</td><td class="r_blue" style="text-align:Right;">450,627,739,550</td>
</tr><tr>
<td class="r_blue" style="text-align:Left;">外國法人</td><td class="r_blue" style="text-align:Rig

# (5.) Extract Content

In [8]:
table_htm = soup.find('table',{ 'id' : 'ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_TableCarte'})

for td in table_htm.find_all('td') : 

    print("---------")
    print(td)
    print(td.text)

---------
<td class="MemDataBlueHeader" style="text-align:Center;">委任人類別</td>
委任人類別
---------
<td class="MemDataBlueHeader" style="text-align:Center;">契約數量</td>
契約數量
---------
<td class="MemDataBlueHeader" style="text-align:Center;">全體有效契約金額</td>
全體有效契約金額
---------
<td class="r_blue" style="text-align:Left;">本國自然人</td>
本國自然人
---------
<td class="r_blue" style="text-align:Right;">247</td>
247
---------
<td class="r_blue" style="text-align:Right;">16,279,451,706</td>
16,279,451,706
---------
<td class="r_blue" style="text-align:Left;">外國自然人</td>
外國自然人
---------
<td class="r_blue" style="text-align:Right;">0</td>
0
---------
<td class="r_blue" style="text-align:Right;">0</td>
0
---------
<td class="r_blue" style="text-align:Left;">本國法人</td>
本國法人
---------
<td class="r_blue" style="text-align:Right;">168</td>
168
---------
<td class="r_blue" style="text-align:Right;">450,627,739,550</td>
450,627,739,550
---------
<td class="r_blue" style="text-align:Left;">外國法人</td>
外國法人
---------
<td clas

# (6.) Clean Soup Function

In [9]:
def clean_soup(soup):

    participate = soup .find('table',{ 'id' : 'ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_TableCarte'}) # locate table place
    df          = pd.DataFrame()

    navigate_column = 0 
    row_list        = []

    for i in participate.find_all('td') :

        row_list.append(i.text)

        series           = pd.Series( row_list)
        navigate_column +=1 

        if navigate_column  == 3 :

            row_list         = []
            df               = df.append(series, ignore_index=True)
            navigate_column  = 0

    return df 

clean_soup(soup)

,0,1,2
0,委任人類別,契約數量,全體有效契約金額
1,本國自然人,247,"16,279,451,706"
2,外國自然人,0,0
3,本國法人,168,"450,627,739,550"
4,外國法人,4,"15,975,533,574"
5,政府機關所屬基金,89,"523,803,474,058"
6,企業職工退休基金,0,0
7,投資型保單,322,"1,037,718,136,362"
8,合計,830,"2,044,404,335,250"
